## RetinaFace

[RetinaFace: Single-stage Dense Face Localisation in the Wild](https://arxiv.org/pdf/1905.00641v2.pdf)

RetinaFace是当前（截止2020年5月11日）在[WIDER Face](http://shuoyang1213.me/WIDERFACE/)测试数据集上准确率最高的人脸识别模型。RetinaFace在学习过程中采用了Face Classification, Face Box Regression, Facial Landmark Regression, 以及像素级的Dense Face Regression（图片见论文Fig. 1）。

本次讨论重点在于论文中4.8部分，推断效率（Inference Efficiency）。在使用轻量级模型的时候，作者用ImageNet训练出来的的模型来初始化前两层卷积层已达到较高的准确率。这里主要关注处理VGA图像（640\*480）的速度，在使用[TVM](https://www.usenix.org/system/files/osdi18-chen.pdf)加速，Intel i7-6700K CPU的一个核心运行，以及MobileNet-0.25模型时，每张VGA图像的处理时间是17.2ms，对应的帧率是58fps。当使用ARM平台（RK3399(A72x2)，在移动端使用）时，每张VGA图像的处理时间是61.2ms，对应的帧率是16fps。这里作者没有定义处理的具体过程，在实际操作的过程中极有可能会得到比作者测试更低的fps。

## InsightFace

[Github](https://github.com/deepinsight/insightface#pretrained-models)

InsightFace是一个对RetinaFace论文思路的应用，支持使用多种底层网路训练和推断（如ResNet50和MobileNet）。

其中，有一个口罩人脸以及面部5点特征识别的项目。原项目检测的是单张图片，在此基础上我加入了视频流的测试，代码如下。

* 人脸概率为0.8，作为在过滤人脸选框时的阈值
* 口罩概率为0.5，在口罩概率大于0.5是人脸选框为绿色（代表佩戴口罩），小于0.5则为红色（代表未佩戴口罩）
* scales是一个输入进模型的变量，让输入的图像以一定比例缩小，提高运行速度。这里使用0.4，降低输入图片尺寸会影响准确率。
* gpu_id设为-1，只是用CPU运行

在没有加入每N帧或者人脸追踪时，运行的帧率约为10-11fps，效果依旧不是十分理想。考虑到作者使用的模型为MobileNet-0.25（这里MobileNet没有后缀，推断为[MobileNetV1](https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet.py)，0.25是MobileNet模型的一个超参数alpha，控制模型的宽度。当alpha低于1时，依据alpha的值来按比例减少每层网络的filter），猜测如果使用最新的[MobilenetV3-Small-Minimalistic-1.0-224](https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v3.py)训练的话，会在运行速度上有一定提升（依据模型介绍中给出的数据，在Pixel1的CPU上运行时，每张图片的处理速度为12.2ms）。
```python
import cv2
import time
from retinaface_cov import RetinaFaceCoV

thresh = 0.8
mask_thresh = 0.5
scales = [0.4]
gpu_id = -1  # do not use GPU

detector = RetinaFaceCoV('./model/mnet_cov2', 0, gpu_id, 'net3l')

vc = cv2.VideoCapture(0)

loop = 0
start = time.time()

while True:
    loop += 1

    ret, frame = vc.read()
    faces, landmarks = detector.detect(frame, thresh, scales=scales)

    if faces is not None:
        text = "{} face(s) found".format(faces.shape[0])
        cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        for face, landmark in zip(faces, landmarks):
            bbox = face[:4]
            # face_prob = face[4]
            mask_prob = face[5]

            # green bounding box for people wearing mask and red for people not wearing mask
            color = (0, 0, 255)
            if mask_prob > mask_thresh:
                color = (0, 255, 0)

            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)

            for x, y in landmark:
                cv2.circle(frame, (x, y), 1, (255, 0, 0), 2)

    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # end timer
        end = time.time()
        print("Average fps: ", loop/(end-start))
        break

# Release handle to the web camera
vc.release()
cv2.destroyAllWindows()
```

在InsightFace的[wiki](https://github.com/deepinsight/insightface/wiki/TVM-Benchmark)里，作者介绍了使用TVM加速和MobileFaceNet(dim128)时在各个平台上的运行效率。这里可以看出，同一张图片使用TVM加速和简单使用MXNet时，TVM加速是的运行速度由28ms提高到了6.4ms。因此，TVM加速也是一个值得尝试的提高fps的方法。

## MobileNetV3 (Small)

[Github: Keras Implementation](https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v3.py)

[Github: TensorFlow Implementation](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet_v3.py)

[Searching for MobileNetV3](https://arxiv.org/pdf/1905.02244.pdf)

MobileNetV3的学习还在进行中，等结束之后补全